In [1]:
#Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
#Create path to draft CSV and convert to DataFrame by reading in the CSV
stats_path = 'resources/QBStats_all.csv'
stats_df = pd.read_csv(stats_path, encoding ='utf8')
stats_df = stats_df.loc[stats_df['year']>=2004]
stats_df.head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year
5097,Peyton ManningP. Manning,29,16,256.0,8.8,2,1,64,1.0,12.0,93.5,24,away,2004
5098,Tom BradyT. Brady,38,26,335.0,8.8,3,1,29,2.0,15.0,111.2,27,home,2004
5099,Matt HasselbeckM. Hasselbeck,29,19,246.0,8.5,1,1,38,0.0,0.0,89.2,21,away,2004
5100,Aaron BrooksA. Brooks,37,18,223.0,6.0,1,1,33,2.0,16.0,65.5,7,home,2004
5101,Carson PalmerC. Palmer,27,18,248.0,9.2,2,1,53t,1.0,10.0,105.2,24,away,2004


In [3]:
qb_stats_df = stats_df
names = qb_stats_df['qb'].tolist()
qb_list = []
for name in names:
    split_name = name.split(".")
    drop_second = split_name[0]
    clean_name = drop_second[:-1]
    qb_list.append(clean_name)
stats_df['Player'] = qb_list
#Create a game count column that will be used later for a weighted average.
game_count = stats_df['Player'].value_counts()
stats_df.head()


,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year,Player
5097,Peyton ManningP. Manning,29,16,256.0,8.8,2,1,64,1.0,12.0,93.5,24,away,2004,Peyton Manning
5098,Tom BradyT. Brady,38,26,335.0,8.8,3,1,29,2.0,15.0,111.2,27,home,2004,Tom Brady
5099,Matt HasselbeckM. Hasselbeck,29,19,246.0,8.5,1,1,38,0.0,0.0,89.2,21,away,2004,Matt Hasselbeck
5100,Aaron BrooksA. Brooks,37,18,223.0,6.0,1,1,33,2.0,16.0,65.5,7,home,2004,Aaron Brooks
5101,Carson PalmerC. Palmer,27,18,248.0,9.2,2,1,53t,1.0,10.0,105.2,24,away,2004,Carson Palmer


In [4]:
stats_df = stats_df.sort_values('Player', ascending=True)
stats_df2 = stats_df[stats_df['Player'] ==""]
combined_names = []
uncleaned_names_list = stats_df2['qb'].tolist()
for uncleaned_name in uncleaned_names_list:
    split_name = uncleaned_name.split()
    split_name.remove(split_name[1])
    combined_name = " ".join(split_name)
    combined_names.append(combined_name)
stats_df.loc[stats_df2.index,'Player'] = combined_names
stats_df.sort_values('Player', ascending=True).head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year,Player
5251,A.J. FeeleyA. Feeley,2,1,6.0,3.0,0,0,6,0.0,0.0,56.2,10,away,2004,A.J. Feeley
10007,A.J. FeeleyA. Feeley,22,12,156.0,7.1,0,1,34,4.0,30.0,58.1,0,away,2011,A.J. Feeley
7458,A.J. FeeleyA. Feeley,42,19,220.0,5.2,1,4,47,1.0,1.0,30.0,24,home,2007,A.J. Feeley
5606,A.J. FeeleyA. Feeley,35,17,170.0,4.9,1,1,21,4.0,26.0,60.4,17,away,2004,A.J. Feeley
5540,A.J. FeeleyA. Feeley,51,25,303.0,5.9,3,5,36,1.0,6.0,47.7,32,home,2004,A.J. Feeley


In [27]:
new_groupby = stats_df.groupby(['Player','year'])['rate'].mean()
qbr_df = pd.DataFrame(columns=('Player','Year','QBR'))
idx = 0
for i in new_groupby.items():
    qbr_df.loc[idx,'Player']= i[0][0]
    qbr_df.loc[idx,'Year']= i[0][1]
    qbr_df.loc[idx,'QBR']= i[1]
    idx +=1
qbr_df

,Player,Year,QBR
0,A.J. Feeley,2004,60.5909
1,A.J. Feeley,2006,106.35
2,A.J. Feeley,2007,64.7
3,A.J. Feeley,2011,60.8
4,A.J. McCarron,2015,93.5167
5,Aaron Brooks,2004,81.275
6,Aaron Brooks,2005,70.8308
7,Aaron Brooks,2006,61.3857
8,Aaron Rodgers,2005,58
9,Aaron Rodgers,2006,59.75


In [30]:
engine = create_engine('sqlite:///nfl_etl.sqlite', echo=False)
qbr_df.to_sql('qb_stats3', con=engine)
engine.execute("SELECT * FROM qb_stats3").fetchall()

[(0, 'A.J. Feeley', 2004, 60.59090909090909),
 (1, 'A.J. Feeley', 2006, 106.35),
 (2, 'A.J. Feeley', 2007, 64.7),
 (3, 'A.J. Feeley', 2011, 60.800000000000004),
 (4, 'A.J. McCarron', 2015, 93.51666666666667),
 (5, 'Aaron Brooks', 2004, 81.27499999999999),
 (6, 'Aaron Brooks', 2005, 70.83076923076923),
 (7, 'Aaron Brooks', 2006, 61.38571428571428),
 (8, 'Aaron Rodgers', 2005, 58.0),
 (9, 'Aaron Rodgers', 2006, 59.75),
 (10, 'Aaron Rodgers', 2007, 103.44999999999999),
 (11, 'Aaron Rodgers', 2008, 94.80624999999999),
 (12, 'Aaron Rodgers', 2009, 104.37499999999999),
 (13, 'Aaron Rodgers', 2010, 98.57333333333332),
 (14, 'Aaron Rodgers', 2011, 121.42666666666666),
 (15, 'Aaron Rodgers', 2012, 105.14375000000001),
 (16, 'Aaron Rodgers', 2013, 103.78888888888888),
 (17, 'Aaron Rodgers', 2014, 113.23124999999999),
 (18, 'Aaron Rodgers', 2015, 93.50625),
 (19, 'Aaron Rodgers', 2016, 105.8875),
 (20, 'Ace Sanders', 2013, 158.3),
 (21, 'Adam Jones', 2005, 0.0),
 (22, 'Adam Podlesh', 2011, 39.6),

In [32]:
import sqlite3
conn = sqlite3.connect('nfl_etl.sqlite')
c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE qb_stats3 RENAME TO old_table;
    
    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE qb_stats4 (idx INTEGER PRIMARY KEY NOT NULL, Player TEXT,
                            Year INTEGER, QBR REAL);

    INSERT INTO qb_stats4 SELECT * FROM old_table;


    DROP TABLE old_table;
    COMMIT TRANSACTION;
    
    PRAGMA foreign_keys=off''')

#close out the connection
c.close()
conn.close()